In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [3]:
gs = pd.read_csv('C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\genome-scores.csv', encoding='ISO-8859-1', low_memory=False)
movies = pd.read_csv('C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\movies.csv', encoding='ISO-8859-1', low_memory=False)
gt = pd.read_csv('C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\genome-tags.csv', encoding='ISO-8859-1', low_memory=False)
ratings = pd.read_csv('C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\ratings.csv', encoding='ISO-8859-1', low_memory=False)
links = pd.read_csv('C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\links.csv', encoding='ISO-8859-1', low_memory=False)
tags = pd.read_csv('C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\tags.csv', encoding='ISO-8859-1', low_memory=False)

In [4]:
gs.head()

,movieId,tagId,relevance
0,1,1,0.03200
1,1,2,0.02225
2,1,3,0.07000
3,1,4,0.05900
4,1,5,0.12300


In [5]:
gs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18472128 entries, 0 to 18472127
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 422.8 MB


In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86537 entries, 0 to 86536
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  86537 non-null  int64 
 1   title    86537 non-null  object
 2   genres   86537 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [8]:
gt.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [9]:
gt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [11]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 1.0 GB


In [12]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [13]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86537 entries, 0 to 86536
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  86537 non-null  int64  
 1   imdbId   86537 non-null  int64  
 2   tmdbId   86411 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.0 MB


In [14]:
tags.head()

,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746


In [15]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2328315 entries, 0 to 2328314
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   userId     int64 
 1   movieId    int64 
 2   tag        object
 3   timestamp  int64 
dtypes: int64(3), object(1)
memory usage: 71.1+ MB


<h3> Создание таблицы popularity для выявления среднего рейтинга на основе ratings

In [28]:
def create_popularity_table():
    movies_path = 'C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\movies.csv'
    ratings_path = 'C:\\Users\\user\\Downloads\\ml-latest\\ml-latest\\ratings.csv'

    movies = pd.read_csv(movies_path, encoding='ISO-8859-1', low_memory=False)
    ratings = pd.read_csv(ratings_path, encoding='ISO-8859-1', low_memory=False)

    avg_ratings = ratings.groupby("movieId")["rating"].mean().reset_index()
    avg_ratings.columns = ["movieId", "ave_rating"]

    popularity = pd.merge(
        movies[["movieId", "title", "genres"]],
        avg_ratings,
        on="movieId"
    )
    return popularity

In [30]:
# Создание таблицы популярности
popularity = create_popularity_table()

<h3> Топ-10 фильмов по популярности в заданном жанре

In [32]:
def get_top_by_genre(genre: str, popularity_df: pd.DataFrame) -> pd.DataFrame:
    # Разделение жанров и фильтрация по заданному
    exploded = popularity_df.assign(genres=popularity_df["genres"].str.split(r"\|")).explode("genres")
    filtered = exploded[exploded["genres"] == genre]
    # Сортировка по рейтингу и выбор топ-10
    top10 = filtered.sort_values("ave_rating", ascending=False).head(10)
    return top10[["title", "genres", "ave_rating"]]

# топ-10 фильмов по жанру 
print(get_top_by_genre("Drama", popularity))

                                                   title genres  ave_rating
72216                            Backyard Wedding (2010)  Drama         5.0
77814                    The Dove's Lost Necklace (1992)  Drama         5.0
54808                            The Jade Pendant (2017)  Drama         5.0
63343                                 The Deadman (1989)  Drama         5.0
31485                                        About Sarah  Drama         5.0
67845                          Story of a Bad Boy (1999)  Drama         5.0
25330                                  Invitation (1952)  Drama         5.0
79526                                    Pendular (2017)  Drama         5.0
77813                     Wanderers of the Desert (1984)  Drama         5.0
60791  All's Well That Ends Well: Shakespeare's Globe...  Drama         5.0


<h3>  Рекомендации по фильму без учета рейтинга

In [42]:
# Преобразование жанров в признаки (TF-IDF)
def get_movie_recommendations(movie_title: str, movies_df: pd.DataFrame) -> list:
    tfidf = TfidfVectorizer(
        tokenizer=lambda x: x.split("|"), 
        token_pattern=None
    )
    genres_matrix = tfidf.fit_transform(movies_df["genres"])
    
    # Прямой расчет схожести (без SVD)
    cosine_sim = cosine_similarity(genres_matrix)
    
    idx = movies_df.index[movies_df["title"] == movie_title].tolist()[0]
    sim_scores = sorted(enumerate(cosine_sim[idx]), key=lambda x: x[1], reverse=True)[1:11]
    return movies_df["title"].iloc[[i[0] for i in sim_scores]].tolist()

In [36]:
# Фильтрация фильмов с высокой популярностью
movie_rating_counts = ratings["movieId"].value_counts()
popular_movies = movie_rating_counts[movie_rating_counts > 100].index
movies_filtered = movies[movies["movieId"].isin(popular_movies)]

In [38]:
print(get_movie_recommendations("Toy Story (1995)", movies_filtered))

['Antz (1998)', 'Toy Story 2 (1999)', 'Adventures of Rocky and Bullwinkle, The (2000)', "Emperor's New Groove, The (2000)", 'Monsters, Inc. (2001)', 'DuckTales: The Movie - Treasure of the Lost Lamp (1990)', 'Wild, The (2006)', 'Shrek the Third (2007)', 'Tale of Despereaux, The (2008)', 'Asterix and the Vikings (AstÃ©rix et les Vikings) (2006)']


<h3>  Рекомендации по фильму с учетом рейтинга